In [2]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import openpyxl
import dbfread
import glob
from pandastable import Table, TableModel # Importar clases de pandastable

In [8]:
#cargar primero el padron de sidefi
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            #padron = pd.read_excel(ruta_archivo)
            padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/8074.csv' mode='r' encoding='cp1252'>
       Nro. OS          Plan del Afiliado Opcion  Nro. Grupo Familiar  \
0            7  Monotributista social EPV     SI             26407599   
1            7  Monotributista social EPV     SI             26407599   
2            7  Monotributista social EPV     SI             26407599   
3           10  Monotributista social MEP     SI             31403090   
4            1                   Afiliado     NO             35995281   
...        ...                        ...    ...                  ...   
15941        4   Monotributista social SD     SI             43194699   
15942        1                   Afiliado     NO             31894447   
15943        1                   Afiliado     NO             42629799   
15944        1                   Afiliado     SI             36602782   
15945        7  Monotributista social EPV     SI             32749818   

         Nro. Afiliado Parentesco c

##### cargar padron de gecros

In [13]:
archivo = None
while True:
    archivo=funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            dbf = dbfread.DBF(ruta_archivo)
            padron = pd.DataFrame(dbf)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='C:/Users/zickd/Downloads/PADRON_GECROS.DBF' mode='r' encoding='cp1252'>


In [9]:
padron.dtypes

Nro. OS                        int64
Plan del Afiliado             object
Opcion                        object
Nro. Grupo Familiar            int64
Nro. Afiliado                 object
Parentesco con el TITULAR     object
Apellido Afiliado             object
Nombre Afiliado               object
Tipo Documento                object
Nro. Documento                 int64
Nro. CUIL                      int64
Fecha Nacimiento              object
Sexo                          object
Nacionalidad                  object
Estado Civil                  object
Legajo Interno                object
Nro. Cuenta                  float64
Fecha Alta                    object
Fecha Baja                    object
Observacion Baja             float64
Tipo Afiliado                  int64
Dependencia                    int64
Observacion                  float64
Calle                         object
Nro. Puerta                    int64
Piso                         float64
Departamento                  object
B

#### cargar el txt  de la sss

In [3]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        columnas = ['Código de Obra Social','CUIT del empleador','CUIL del Titular','Código de Parentesco','Cuil','Tipo Documento',	'Número Documento',	'Apellido y Nombre','Sexo',	'Estado Civil','Fecha de Nacimiento',
                    'Nacionalidad',	'Calle','Número de puerta',	'Piso',	'Departamento',	'Localidad','Código Postal','Provincia','Tipo de Domicilio','Teléfono',	'Situación revista','Incapacidad','Tipo de beneficiario titular','Fecha de alta en la Obra Social',
                    'Fecha de cierre de la presentación','Código de Movimiento','Cod Error1','Cod Error2','Cod Error3']
        try:
            devolucionSss = pd.read_csv(archivo, sep="|", encoding="utf-8", names = columnas )
            #devolucionSss[['Cod Error1','Cod Error2','Cod Error3']]=devolucionSss[['Cod Error1','Cod Error2','Cod Error3']].replace('-', ' ')
            print(devolucionSss)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='//192.168.2.110/auditoria medica/sss salud PADRONES/padron/ftp novedades padron/2024/abril2024/127406-202402.err' mode='r' encoding='cp1252'>


ParserError: Too many columns specified: expected 30 and found 28

In [11]:
devolucionSss.dtypes

Código de Obra Social                  int64
CUIT del empleador                     int64
CUIL del Titular                       int64
Código de Parentesco                   int64
Cuil                                   int64
Tipo Documento                        object
Número Documento                       int64
Apellido y Nombre                     object
Sexo                                  object
Estado Civil                           int64
Fecha de Nacimiento                    int64
Nacionalidad                           int64
Calle                                 object
Número de puerta                       int64
Piso                                   int64
Departamento                          object
Localidad                             object
Código Postal                         object
Provincia                              int64
Tipo de Domicilio                      int64
Teléfono                              object
Situación revista                      int64
Incapacida

#### diccionario de errores

In [14]:
errores = {
"0":"Novedad aceptada. Sin errores de validación.",
"1":"CUIT del Empleador. Debe estar informado",
"2":"CUIT del Empleador. La longitud debe ser de 11 caracteres y debe ser numérico",
"3":"CUIT del Empleador. Se verifica que el prefijo sea 30, 33, 34,  20, 27, 23 o 24. Para Código de Movimiento A, M.",
"4":"CUIT del Empleador. Se verifica que el dígito verificador sea correcto. Para Código de Movimiento A, M.",
"5":"CUIL del Titular. Debe ser informado",
"6":"CUIL del Titular. La longitud debe ser de 11 caracteres y debe ser numérico",
"7":"CUIL del Titular. Se verifica que el prefijo sea 20, 27, 23 o 24. Para Código de Movimiento A, M.",
"8":"CUIL del Titular. Se verifica que el dígito verificador sea correcto. Para Código de Movimiento A, M.",
"9":"Parentesco. Debe ser informado",
"10":"Parentesco. Máximo de 2 posiciones numéricas",
"11":"Parentesco. Debe existir en la tabla correspondiente",
"12":"CUIL. Debe ser informado.",
"13":"CUIL. La longitud debe ser de 11 caracteres y debe ser numérico.",
"14":"CUIL. Se verifica que el prefijo sea 20, 27, 23 o 24",
"15":"CUIL. Se verifica que el dígito verificador sea correcto. Para Código de Movimiento A, M.",
"16":"Tipo de Documento. Debe ser informado",
"17":"Tipo de Documento. Máximo de 2 posiciones",
"18":"Tipo de Documento. Debe existir en la tabla correspondiente",
"19":"Nro. De Documento. Debe ser informado",
"20":"Nro. De Documento. Máximo de  8 posiciones numéricas",
"21":"Nro. De Documento. No se aceptan números como 11111111, 22222222, 33333333, etc",
"22":"Apellido y Nombres. Debe ser informado",
"23":"Apellido y Nombres. De 30 posiciones como máximo",
"24":"Sexo. Debe ser informado. 1 posición",
"25":"Sexo. Debe existir en la tabla correspondiente",
"26":"Estado Civil. Debe ser informado",
"27":"Estado Civil. Máximo de 2 posiciones numéricas",
"28":"Estado Civil. Debe existir en la tabla correspondiente",
"29":"Fecha de Nacimiento. Debe ser informado",
"30":"Fecha de Nacimiento. Numérico de 8 posiciones",
"31":"Fecha de Nacimiento. No puede ser posterior a la fecha del día al momento de realizarse el proceso",
"32":"Fecha de Nacimiento. La edad no puede ser mayor a 110 años",
"33":"Fecha de Nacimiento. Debe ser una fecha válida con formato DDMMAAAA",
"34":"Nacionalidad. Debe ser informado",
"35":"Nacionalidad. Máximo de 3 posiciones numéricas",
"36":"Nacionalidad. Debe existir en la tabla correspondiente",
"37":"Calle. Máximo de  20 posiciones",
"38":"Número de puerta. Máximo de 5 posiciones",
"39":"Piso. Máximo de 4 posiciones",
"40":"Departamento. Máximo de 4 posiciones",
"41":"Localidad. Máximo de 20 posiciones",
"42":"Código Postal. Máximo de 8 posiciones",
"43":"Provincia. Debe ser informado",
"44":"Provincia. Numérico de 2 posiciones",
"45":"Provincia. Debe existir en la tabla correspondiente",
"46":"Tipo de Domicilio. Debe ser informado",
"47":"Tipo de Domicilio. Máximo de  2 posiciones numéricas",
"48":"Tipo de Domicilio. Debe existir en la tabla correspondiente",
"49":"Situación de revista. Debe ser informado",
"50":"Situación de revista. Máximo de  2 posiciones numéricas",
"51":"Situación de revista. Debe existir en la tabla correspondiente",
"52":"Incapacidad. Debe ser informado",
"53":"Incapacidad. Numérico de 2 posiciones",
"54":"Incapacidad. Los valores aceptados son 00 ó 01",
"153":"El Código de Movimiento debe ser informado. A, B o M.",
"168":"No se permiten novedades de cambio de CUIIL, si el Código de Movimiento es C.",
"55":"Tipo de beneficiario titular. Debe ser informado",
"56":"Tipo de beneficiario titular. Numérico de 2 posiciones",
"57":"Tipo de beneficiario titular. Debe existir en la tabla correspondiente.",
"58":"Fecha de Ingreso. Debe ser informado",
"59":"Fecha de Ingreso. Numérico de 8 posiciones",
"60":"Fecha de Ingreso. Debe ser una fecha válida con formato DDMMAAAA.",
"61":"CUIT del Empleador - Tipo de Beneficiario- CUIL  del Titular. Para Tipos de Beneficiarios 00, 01 y 12 Relación de Dependencia, Pasante y Relación de Dependencia sin aportes y contrib por SIJP respectivamente, el CUIT del Empleador no puede ser el CUIL del Titular",
"62":"CUIT del Empleador - Tipo de Beneficiario- CUIL  del Titular. Para Tipo de Beneficiario 02, 08 y 10,  Jubilado y Pensionado dentro del Sistema, Desempleo y Jubilados y Pensionado de  Pami el CUIT debe ser el de ANSES (33-63761744-9).",
"63":"CUIT del Empleador - Tipo de Beneficiario- CUIL  del Titular. Para Tipos de Beneficiarios 04, 05, 07 y 11, Monotributista Autónomo, Servicio Doméstico, Monotributista Efector Social  y Jubilado y Pensionado fuera del Sistema, respectivamente, el CUIT debe ser el CUIL del Titular.",
"154":"CUIT del Empleador - Tipo de Beneficiario- CUIL  del Titular. Para Tipos de Beneficiarios 09, Desempleo de RENATRE CUIT debe ser el de RENATRE (30-707933333-6)",
"155":"CUIT del Empleador - Tipo de Beneficiario- CUIL  del Titular- Situación de Revista. Para Tipo de Beneficiario 03 y SR 99 Son los socios directos, el CUIT debe ser el CUIL de TITULAR",
"156":"CUIT del Empleador - Tipo de Beneficiario- CUIL  del Titular- Situación de Revista. Para Tipo de Beneficiario 03 (socios directos), el CUIT es el CUIL de TITULAR la SR debe ser 99.",
"157":"CUIT del Empleador - Tipo de Beneficiario- CUIL  del Titular- Situación de Revista. Para Tipo de Beneficiario 12 y SR 98 Son los exceptuados, el CUIT no debe ser el CUIL de TITULAR.",
"158":"CUIT del Empleador - Tipo de Beneficiario- CUIL  del Titular- Situación de Revista. Para Tipo de Beneficiario 12 (Exceptuados), el CUIT no es el CUIL de TITULAR la SR debe ser 98.",
"159":"Tipo de Beneficiario – Situación de Revista. Para Tipo de Beneficiario 08 la Situación de Revista es 13",
"160":"Tipo de Beneficiario – Situación de Revista. Para Tipos de Beneficiarios 00, 01, 02, 04,05, 07, 09, 10 , 11 y 12  la Situación de Revista es diferente a 50, 51, 52, 53, 13 y 98. Para Tipo de Beneficiario 06 la Situación de Revista es 50, 51, 52, 53,",
"169":"Tipo de Beneficiario- Situación de Revista La Situación de Revista es 70 (Amparo Judicial) , sólo si el Tipo de Beneficiario es 11.",
"161":"Tipo de Beneficiario – Código de Obra Social. Para Tipo de beneficiario 09 el código de Obra Sociales es 119302",
"162":"Tipo de Beneficiario – Código de Obra Social. Para Tipo de beneficiario 10 el código de Obra Sociales es 500807",
"165":"Tipo de Beneficiario – Código de Obra Social.Baja de Beneficiario. Para Tipo de Beneficiario 3 (adherente voluntario)  la Obra Social no está inscripta para aceptar estos beneficiarios.",
"163":"Situación de Revista – Parentesco. Para Situación de Revista 53 y Tipo de Beneficiario 06 t el CUIT es el de Anses o Salud, el código de parentesco es 01.",
"64":"Fecha de Nacimiento - Fecha de Ingreso. La fecha de nacimiento no puede ser mayor a la fecha de Ingreso a la Obra Social",
"65":"CUIL (del Beneficiario) - CUIL  del Titular. Se trata de una Familiar, el CUIL del Titular no puede ser igual al CUIL.",
"66":"Parentesco - Fecha de Nacimiento. Para parentescos 3 y 5, Hijo soltero menor de 21 años e Hijo del cónyuge soltero menor de 21 años, la edad debe ser menor a 21 años",
"67":"Parentesco - Fecha de Nacimiento. Para parentescos 4 y 6, Hijo soltero de 21 a 25 años cursando estudios regulares e Hijo del cónyuge soltero de 21 a 25 años cursando estudios regulares, la edad debe ser mayor a 21 y menor a 25 años",
"68":"Parentesco - Fecha de Nacimiento. Para parentesco 7, Menor bajo guarda o tutela, la edad debe  ser menor a 25 años",
"69":"Parentesco - Fecha de Nacimiento. Para parentesco 9, Mayor de 25 años discapacitado, la edad debe ser mayor o igual a 25 años",
"70":"Parentesco - Incapacidad. Para parentesco 9 Mayor de 25 años discapacitado, el campo de incapacitado debe estar informado con 01",
"71":"Situación de Revista - Sexo. Para Situación de Revista 10 Maternidad, el Sexo debe ser Femenino, F",
"72":"Tipo de documento - Nacionalidad. Para extranjeros solo se permite: Documento Único, Pasaporte, Certificado Migratorio o Cédula de Identidad",
"73":"Tipo de documento - Nacionalidad. Para argentinos no se acepta Pasaporte ni Certificado Migratorio",
"74":"Tipo de documento - Nacionalidad. Para argentinos no se aceptan cédulas cuando el CUIL no está informado, ya que no sería posible cuilificar",
"75":"Tipo de documento - Sexo. Para Libreta Cívica le corresponde Sexo Femenino, F",
"76":"Tipo de documento - Sexo. Para Libreta de Enrolamiento Sexo Masculino, M",
"77":"Parentesco - Estado Civil. Si es Soltero 1, no puede ser Cónyuge 1",
"78":"Parentesco - Estado Civil. Si es Casado 2, Viudo 3, Separado Legal 4, Separado de Hecho 5, Divorciado 6,  no puede ser  3 al 7, Hijo Soltero",
"79":"Se rechaza el registro debido a que el Familiar (CUIL del Titular) no posee su correspondiente Titular (CUIL)",
"80":"Titulares duplicados (CUIL Y CUIT DEL EMPLEADOR)",
"81":"Familiares duplicados dentro de un mismo grupo",
"141":"Cambio de grupo familiar efectuado por un reclamo presentado en la SSSalud por parte de un beneficiario. Y Estos reclamos generan un memo o un expediente dictaminado que autoriza dicho movimiento.",
"82":"Fecha de Ingreso - Fecha de Presentación. La fecha de Ingreso a la Obra Social no puede ser posterior a la fecha de presentación del Padrón.",
"83":"Fecha de Presentación. Debe ser informado",
"84":"Fecha de Presentación. Numérico de 8 posiciones",
"85":"Fecha de Presentación. Debe ser una fecha válida con formato DDMMAAAA.",
"88":"Código de movimiento erróneo debe ser A (Alta), B (Baja), M (Modificación) y C (Cambio de Cuil)",
"89":"Novedad Duplicada",
"90":"Para movimiento de baja no se encuentra registro.",
"91":"Para movimiento de modificación no se encuentra registro.",
"92":"Alta existente Para Titulares: igual CUIL, CUIT y OBRA SOCIAL Para Familiares : CUIL, CUIT, OBRA SOCIAL, TIPO DOC, NRO DOC, PRIMERAS 5 LETRAS DEL APELLIDO",
"93":"No existe titular para el familiar informado en la novedad.",
"94":"Para movimiento de baja, se encuentra más de un registro. No coincide el CUIT para beneficiario con pluriempleo.",
"95":"Para movimiento de alta de familiar, familiar existente sin CUIL. Se tomó el CUIL informado en la novedad.",
"96":"Para movimiento de modificación, se encuentra más de un registro. No coincide el CUIT para beneficiario con pluriempleo",
"97":"Para novedad de cambio de CUIL no se encuentra el registro.",
"98":"El tipo de documento ET solo se permite para parentescos 3, 5 o 7",
"99":"Para tipo de documento ET el número de documento no se corresponde con la parte central del cuil del titular.",
"110":"Para pensionado español: CUIT de empleador inválido",
"111":"Beneficiario español  no informado por el Ministerio de Trabajo y Asuntos Sociales del Reino de España",
"112":"Obra social no habilitada para recibir pensionados españoles",
"113":"El pensionado español tiene una opción a otra obra social de jubilados",
"114":"Para pensionados españoles la situación de revista debe ser válida, 50, 51, 52 o 53",
"116":"La nacionalidad no es española.",
"117":"Para situación de revista 50 o 52 el CUIT debe ser 33637617449",
"118":"Para situación de revista 51 el CUIT debe ser 30694407044",
"119":"Para situación de revista 53, parentesco debe ser 1 y el CUIT debe ser Anses o SSSalud.",
"120":"Para situación de revista 50, 51, 52, parentesco debe ser 0",
"121":"Para pensionados españoles: el pensionado se encuentra en otra obra social distinta a la que lo está declarando.",
"129":"Para tipo de documento ET la fecha de nacimiento no puede superar el año calculado con la fecha de relevamiento.",
"130":"Para tipo de documento ET el CUIL debe ser el CUIL del TITULAR seguido de la parte central del cuil del titular.",
"131":"No se acepta tipo de documento ET para movimientos M o C",
"132":"No se acepta código de movimiento C sin cuil en el campo CUIL.",
"133":"Para un Alta o Modificación de cónyuge, existe cónyuge en el grupo. De aplicarse la novedad se generarían dos registros con código de parentesco 1 en el mismo grupo familiar Para un Alta o Modificación de concubina, existe concubina en el grupo. De aplicarse la novedad se generarían dos registros con código de parentesco 2 en el mismo grupo familiar.",
"170":"Para Alta de familiares de un mismo grupo familiar (excluyendo Esposo/a y Concubina/o) supera las cantidad de 20 personas.",
"134":"Para registro con código de movimiento C, existe registro. El cambio de cuil ya había sido aplicado en el Padrón de la SSS",
"135":"No existe registro para aplicar el movimiento de cambio de Cuil en el Padrón.",
"136":"La fecha de cierre de la presentación es anterior a la fecha de proceso menos 2 meses. (Sólo se aceptan fechas de presentación de hasta dos meses anteriores a la fecha de actualización).",
"137 (Proximamente)":"La fecha de alta en la Obra Social es posterior al último día del mes de proceso. No se pueden entregar registros con altas posteriores al mes del proceso.",
"139 (Próximamente)":"El código de la Obra Social informado en la primer columna del archivo de novedades no coincide con el número de la Obra Social.",
"138":"El CUIL es inválido.",
"140":"Para Tipo de Beneficiario=0 no tiene DDJJ en los últimos 6 meses.",
"142":"Tipo de Beneficiario=9 y Código de Obra Social=119302. El titular no posee para ese periodo una prestación por desempleo informada por el RENATRE.",
"164":"Para  ALTA, MODIFICACION o CAMBIO DE CUIL y TIPO DE BENEFICIARIO=0 el CUIL DEL TITULAR debe ser igual al CUIL.",
"167":"El código de Obra Social que figura en el registro no coincide con  el código de la carpeta del FTP.",
"63":"Para Tipo de Beneficiario 3,4,5,7,11 el CUIT debe ser igual al CUIL DE TITULAR",
"100":"CUIL pertene a la persona.",
"101":"Registro de CUIL apropiado para titular. La OS informó un Cuil erróneo o Cuil anterior. Se modifica el CUIL y CUIL del Titular incluyendo el grupo familiar",
"104":"CUIL y NRO. DE DOCUMENTO coincidentes, difiere el APELLIDO Y NOMBRE.",
"105":"CUIL coincidente, APELLIDO Y NOMBRE no coincidente al 100%, difiere el NRO DE DOCUMENTO.",
"106":"CUIL coincidente, difieren el NRO. DE DOCUMENTO y APELLIDO Y NOMBRE.",
"107":"CUIL, NRO. DE DOCUMENTO, APELLIDO Y NOMBRE coinciden, difiere el AÑO DE LA FECHA DE NACIMIENTO",
"111":"Registro de CUIL apropiado para familiar. La OS informó un Cuil erróneo o Cuil anterior. Se modifica el CUIL de un familiar",
"112":"Se apropia NRO. DE DOCUMENTO, coinciden el CUIL, APELLIDO Y NOMBRE y SEXO",
"300":"Persona inexistente en el UNIVERSO CUIT- CUIL de AFIP."
}
obraSocial = {
1:"osoetsylra",
2:"otros",
3:"visitar",
4:"SD",
5:"tis",
6:"gerensal",
7:"evp",
10:"meplife"
}

padron['NRODOC']=padron['NRODOC'].fillna(0).astype("Int64")
padron['DEPBEN_NOM']=padron['DEPBEN_NOM'].astype("str")
padron.rename(columns={"DEPBEN_NOM":"Sede"}, inplace=True)
print(padron[['NRODOC','Sede']].head())

sedes = {
"10" : "CABA",
"20" : "ZONA NORTE Y NOROESTE",
"30" : "ZONA OESTE",
"40" : "ZONA SUR",
"50" : "LA PLATA",
"60" : "MAR DEL PLATA",
"70" : "PERGAMINO",
"80" : "ROSARIO",
"90" : "CORDOBA",
"100" : "MENDOZA",
"110" : "TUCUMAN"
}


     NRODOC Sede
0  10000010   10
1  10000353   10
2  10000494   10
3  10001693   10
4  10003713   10


In [15]:
print(devolucionSss.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276 entries, 0 to 1275
Data columns (total 30 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Código de Obra Social               1276 non-null   int64 
 1   CUIT del empleador                  1276 non-null   int64 
 2   CUIL del Titular                    1276 non-null   int64 
 3   Código de Parentesco                1276 non-null   int64 
 4   Cuil                                1276 non-null   int64 
 5   Tipo Documento                      1276 non-null   object
 6   Número Documento                    1276 non-null   int64 
 7   Apellido y Nombre                   1276 non-null   object
 8   Sexo                                1276 non-null   object
 9   Estado Civil                        1276 non-null   int64 
 10  Fecha de Nacimiento                 1276 non-null   int64 
 11  Nacionalidad                        1276 non-null   int6

#### joinear los 2 dataframes

In [16]:

resultado = pd.merge(padron, devolucionSss, left_on = 'NRODOC', right_on = 'Número Documento', how="right")
resultado[["Cod Error1", "Cod Error2", "Cod Error3"]] = resultado[["Cod Error1", "Cod Error2", "Cod Error3"]].astype(str)
resultado[['Cod Error1','Cod Error2','Cod Error3']] = resultado[['Cod Error1','Cod Error2','Cod Error3']].replace(errores)
resultado['Sede'] = resultado['Sede'].astype(str).replace(sedes)
resultado['Sede'] = resultado['Sede'].where(resultado['Sede'] != "<NA>", "REVISAR")
resultado['OS_NOMBRE'] = resultado['OS_NOMBRE'].fillna("REVISAR")



resultado = resultado[[
"Sede",
"OS_NOMBRE",
"Código de Obra Social",
"CUIT del empleador",
"CUIL del Titular",
"Código de Parentesco",
"Cuil",
"Número Documento",
"Apellido y Nombre",
"Fecha de Nacimiento",
"Localidad",
"Provincia",
"Tipo de beneficiario titular",
"Fecha de alta en la Obra Social",
"Código de Movimiento",
"Cod Error1",
"Cod Error2",
"Cod Error3"
]]


In [17]:

print(resultado['Sede'].dtypes)
resultado.to_excel('resiltadofinal.xlsx',index=False)


object


### Armar los excels segun sede

In [18]:
groups = resultado.groupby('Sede') # Agrupar el dataframe por la columna convenio
dfs = [] # Crear una lista vacía para almacenar los dataframes separados
for name, group in groups: # Iterar sobre los grupos y añadir cada dataframe a la lista
    dfs.append(group)
    print(dfs) # Imprimir la lista de dataframes

save_path = filedialog.askdirectory()
devolucionSss.to_excel(save_path + '\devoluciones2_septiembre2023.xlsx', engine='openpyxl' ,index = False )

### guardarlos con los nombres correspondientes
for df in dfs: # Iterar sobre la lista de dataframes
    sede = df['Sede'].iloc[0] # Obtener el nombre del convenio del primer elemento del grupo
    filename = save_path + '/' + sede + '.xlsx' # Crear el nombre del archivo con la ruta de la carpeta y el nombre del convenio
    df.to_excel(filename, index=False) # Guardar el dataframe en el archivo excel sin el índice
    print(f"Se ha guardado el archivo {filename}") # Imprimir un mensaje de confirmación

[      Sede   OS_NOMBRE  Código de Obra Social  CUIT del empleador  \
0     CABA  OSOETSYLRA                 127406         20042970000   
3     CABA  OSOETSYLRA                 127406         20045232752   
4     CABA  OSOETSYLRA                 127406         20050824269   
6     CABA  OSOETSYLRA                 127406         20079666034   
7     CABA  OSOETSYLRA                 127406         20079666034   
...    ...         ...                    ...                 ...   
1262  CABA  OSOETSYLRA                 127406         33655524679   
1263  CABA  OSOETSYLRA                 127406         33655524679   
1264  CABA  OSOETSYLRA                 127406         33655524679   
1265  CABA  OSOETSYLRA                 127406         33655524679   
1266  CABA  OSOETSYLRA                 127406         33655524679   

      CUIL del Titular  Código de Parentesco         Cuil  Número Documento  \
0          27942485579                     0  27942485579          94248557   
3          2

### Alternativa sin cruce

In [ ]:


#resultado = pd.merge(padron, devolucionSss, left_on = 'Nro. Documento', right_on = 'Número Documento', how="right")
resultado[["Cod Error1", "Cod Error2", "Cod Error3"]] = resultado[["Cod Error1", "Cod Error2", "Cod Error3"]].astype(str)
resultado[['Cod Error1','Cod Error2','Cod Error3']] = resultado[['Cod Error1','Cod Error2','Cod Error3']].replace(errores)


In [ ]:
save_path = filedialog.askdirectory()
resultado.to_excel(save_path + '\devoluciones2_octubre2023.xlsx', engine='openpyxl' ,index = False )

In [ ]:
resultado.dtypes